   # Computing Fingerings

## CLASSES

In [ ]:
import xml.etree.ElementTree as ET
from xml.dom.minidom import parse, parseString
from Options import *
from time import sleep
from collections import OrderedDict
import gc
import os.path
import sys

# ============================= ============================= ============================= #
# ============================= ============================= ============================= #  
class Note(object):
    # ============================= ============================= #
    def __init__(self, pitch, octave, voice=None, duration=None, x=None, y=None, measure=0):
        self.pitch = pitch
        self.octave = octave
        self.voice = voice
        self.x = x
        self.y = y
        self.duration = duration
        self.measure = measure
        self.me = str('{0}-{1}'.format(self.pitch, self.octave))
        self.xy = str('{0}-{1}({2}: {3} {4})'.format(self.pitch, self.octave, self.measure, self.x, self.y))
        self.fingers = None
    
    def setFingers(self, fingers):
        self.fingers = fingers
        
    def addFinger(self, finger):
        self.fingers.add(finger)
    
    def getPitch(self):
        return self.pitch
    
    def getOctave(self):
        return self.octave
    
    def getFingers(self):
        return self.fingers
    
    def getVoice(self):
        return self.voice
    
    # show pitch, octave, possible fingerings
    def getNoteInfo(self):
        return ('Note {0} of octave {1} playable at fret-string-finger combination(s): {2}'.format(self.pitch, self.octave, self.fingers))

    # ============================= ============================= #
    # overrides 
    def __eq__(self, other):
        if isinstance(other, Note):
            return (self.pitch == other.pitch) and (self.octave == other.octave)
        else:
            return NotImplemented
        
    def __ne__(self, other):
        res = self.__eq__(other)
        if res is NotImplemented:
            return res
        return not res

    def __lt__(self, other):
        if isinstance(other, Note):
            if float(self.measure) == float(other.measure):
                return float(self.x) < float(other.x)
            else:
                return float(self.measure) < float(other.measure)
        else:
            return NotImplemented
        
    def __rt__(self, other):
        if isinstance(other, Note):
            if float(self.measure) == float(other.measure):
                return float(self.x) > float(other.x)
            else:
                return float(self.measure) > float(other.measure)
        else:
            return NotImplemented
        
    def __le__(self, other):
        if isinstance(other, Note):
            if float(self.measure) == float(other.measure):
                return float(self.x) <= float(other.x)
            else:
                return float(self.measure) <= float(other.measure)
        else:
            return NotImplemented
        
    def __re__(self, other):
        if isinstance(other, Note):
            if float(self.measure) == float(other.measure):
                return float(self.x) >= float(other.x)
            else:
                return float(self.measure) >= float(other.measure)
        else:
            return NotImplemented
    
# ============================= ============================= ============================= # 
# ============================= ============================= ============================= #  
class Chord(object):
    # ============================= ============================= #
    def __init__(self, notes):
        self.notes = notes # list of notenodes 
        self.me = ','.join(str(note.me) for note in self.notes)
        
    def addNote(self, note):
        self.notes.append(note)
                 
    def getNotes(self):
        return self.notes
    
# ============================= ============================= ============================= #
# ============================= ============================= ============================= #           
class NoteNode(object):
    # ============================= ============================= #
    def __init__(self, notes, rules, isChord=False, finger=None, parent=None, fromChord=False):
        self.rules = rules
        if isChord:
            # create nodes for each note in chord, get score and best fignering and keep it as a whole
            # find optimal path first and keep it
            chords = list()
            i = 0
            # initialise with first note
            if len(notes[0].fingers.split(','))>1:
                for finger in notes[0].fingers.split(','):
                    node = NoteNode(notes[0], rules, False, finger, parent)
                    #node.resetScore()
                    chords.append(node)
            else:
                node = NoteNode(notes[0], rules, False, notes[0].fingers, parent)
                #node.resetScore()
                chords.append(node)    
            for note in notes[1:]:
                c = 0
                for child in chords[-i:]:
                    ch_finger = int(str(child.finger).split(',')[-1])
                    if len(note.fingers.split(','))>1:
                        for finger in note.fingers.split(','):
                            if ((finger != ch_finger) or (ch_finger==0)):
                                node = NoteNode(note, rules, False, finger, child, fromChord=True)
                                chords.append(node)
                                c += 1
                    else:
                        if ((note.fingers != ch_finger) or (ch_finger==0)):
                            node = NoteNode(note, rules, False, note.fingers, child, fromChord=True) 
                            chords.append(node)
                            c += 1
                i = c
            chords_leafs = chords[-i:]
            scores = {}
            for leaf in chords_leafs: #get the best possible path for the chord
                scores[leaf] = leaf.score
            best_path = list()
            start = min(scores, key=scores.get)
            best_pitch = ''
            best_fingers = ''
            best_comb = ''
            for note in notes:
                while not start.equals(notes[0]): 
                    best_path.insert(0, start)
                    best_pitch = start.pitch + ',' + best_pitch
                    best_fingers = str(start.finger) + ',' + best_fingers
                    best_comb = '({0},{1})'.format(str(start.fret),str(start.string))+ ' ' + best_comb
                    start = start.parent
            # last note 
            best_path.insert(0, start)
            best_pitch = start.pitch + ',' + best_pitch
            best_fingers = str(start.finger) + ',' + best_fingers
            best_comb = '({0},{1})'.format(str(start.fret),str(start.string))+ ' ' + best_comb
            # got best chord fingerings, now fill the rest 
            self.pitch = best_pitch[:-1]
            self.fret = chords_leafs[-1].fret
            self.string = chords_leafs[-1].string
            self.finger = best_fingers[:-1]
            self.children = list()
            self.octave = chords_leafs[-1].octave
            self.position = chords_leafs[-1].position
            self.note = notes
            if parent:
                self.parent = parent
                self.parent.addChild(self)
            else:
                self.parent = self
            self.score = self.parent.score #+ chords_leafs[-1].score  
            #self.me = str('{0}: {1}: {2}'.format(self.pitch, best_comb[:-1], self.finger))
            self.me = best_comb[:-1]
        else:
            self.fromChord = fromChord
            self.note = notes
            self.vector = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0] # num of rules
            # normal node with 1 note
            self.pitch, self.fret, self.string, self.finger = self.analyze(notes, finger)
            self.score = 0
            self.octave = notes.getOctave()
            self.children = list()
            self.position = self.findPos()
            if parent:
                self.parent = parent
                self.parent.addChild(self)
            else:
                self.parent = self
            #self.me = str('{0}-{1}: ({2}/{3}): {4}'.format(self.pitch, self.octave, self.fret, self.string, self.finger))
            self.me = '({0},{1})'.format(self.fret,self.string)
            self.score = self._score() + self.parent.score
            
    # ============================= ============================= #
    # Note format: (fret/string):finger
    def analyze(self,note,f):
        pitch = note.getPitch()
        if len(f)==7:
            fret = f[1]
            string = f[3]
            finger = f[6]
        else:
            fret = f[1:3]
            string = f[4]
            finger = f[7]
        return pitch, int(fret), int(string), int(finger)
    
    def findPos(self):
        if self.fret == 0:
            return 0
        if self.finger <= self.fret:
            return self.fret - self.finger +1
        else:
            return 1
    
    # ============================= ============================= #
    def _score(self):
        # calculate score
        # the bigger the score the worse the fingering
        #total = 0
        if self.position == self.parent.position: # favor no position change
            self.vector[0] = 1
        if (self.fret-self.position)+1 == self.finger: # prioritize finger-position
            #total += -10
            self.vector[1] = 1
        if self.fret == 0: # prioritize 0
            #total += -20
            self.vector[2] = 1
        if (self.finger != 0) and (self.finger == int(str(self.parent.finger).split(',')[-1])): # fingers should be different
            #total += 10
            self.vector[3] = 1
        if (self.equals(self.parent)) and (self.finger == self.parent.finger): # same finger on holds
            self.vector[4] = 1
        if self.finger < int(str(self.parent.finger).split(',')[-1]): # higher fingers first
            self.vector[5] = 1
        if self.position <= 5: # prioritize smaller positions
            #total += -5
            self.vector[6] = 1
        if self.finger == 4: # penalize 4
            self.vector[7] = 1
        self.vector[8] = abs(self.position - self.parent.position)
        self.vector[9] = abs(self.fret - self.parent.fret)
        self.vector[10] = abs(self.string - self.parent.string) # same as 12?
        self.vector[11] = abs(self.finger - int(str(self.parent.finger).split(',')[-1]))    
        
        if self.fromChord: # only for chords 
            if self.parent.string == self.string: # not feasible
                self.vector[12] = 1
            if (self.fret == self.parent.fret) and (self.finger == self.parent.finger) and (self.finger == 1):
                self.vector[14] = 1
        if self.position < 2: # extra favor 1st pos
            self.vector[13] = 1
        
        return sum(int(a)*int(b) for a,b in zip(self.rules,self.vector))    
            
        #total += abs(self.position - self.parent.position)
        #total += abs(self.fret - self.parent.fret)
        #total += abs(self.string - self.parent.string)
        #total += abs(self.finger - int(str(self.parent.finger).split(',')[-1]))
        #return total
    # ============================= ============================= #
    # Node stuff
    def hasChildren(self):
        return self.children
    
    def isChild(self):
        return self.parent and (self in self.parent.children)
    
    def isRoot(self):
        return (self is self.parent)

    def isLeaf(self):
        return not self.children

    def addChild(self, child):
        if not (child in self.children):
            self.children.append(child)
    
    def resetScore(self):
        self.score = 0
        
    def replaceNodeData(self, notenode):
        self.vector = notenode.vector
        self.pitch = notenode.pitch
        self.fret = notenode.fret
        self.string = notenode.string
        self.finger = notenode.finger
        self.score = notenode.score
        self.octave = notenode.octave
        self.position = notenode.position
        self.me = notenode.me
        
        notenode.parent.addChild(self)
        self.delete()
        self.parent = notenode.parent
        
    def delete(self):
        if self in self.parent.hasChildren():
            self.parent.children.remove(self)
    # ============================= ============================= #
    # check if two nodes contain the same pitch/octave info == are the same
    def equals(self, note):
        return ((note.pitch == self.pitch) and (note.octave == self.octave))

    # ============================= ============================= #
    # overrides 
    def __lt__(self, other):
        if isinstance(other, NoteNode):
            return float(self.score) < float(other.score)
        else:
            return NotImplemented
        
    def __rt__(self, other):
        if isinstance(other, NoteNode):
            return float(self.score) > float(other.score)
        else:
            return NotImplemented
        
    def __le__(self, other):
        if isinstance(other, NoteNode):
            return float(self.score) <= float(other.score)
        else:
            return NotImplemented
        
    def __re__(self, other):
        if isinstance(other, NoteNode):
            return float(self.score) >= float(other.score)
        else:
            return NotImplemented 
        
    # ============================= ============================= #
    # prints
    def getChildren(self):
        child = ''
        for ch in self.children:
            child += ch.me
            child += ' , '
        return child[:-2]
    
    def getNodeInfo(self):
        return ('Pitch {0} on fret {1} of string {2} with finger {3}'.format(self.pitch, self.fret, self.string, self.finger))

# ============================= ============================= ============================= #  
# ============================= ============================= ============================= #  
class NoteTree(object):
    # ============================= ============================= #
    def __init__(self, file, rules, limit=5, pr=True):
        self.options = options # imported
        self.rules = rules # imported in gui
        self.limit = limit # width limitation
        self.root = None # init
        self.fingering_options = OrderedDict() # init
        self.best_path = [] # init
        self.best_dict = OrderedDict() # try
        self.fingering_paths = set() # init
        self._fingering_paths = set() # init
        self.leafs = [] # init
        self.scores = {} # init
        self.new_file = ''
        self.prints = ''
        self.file = ''
        self.combs = '' # for the fret-string tablature combinations

        if self.validate(file):
            self.file = file
            self.fingering_options = self.fillOptions() # created all possible fingering placement combs
            self.note_list = list(self.fillNotes()) # got note list
            self.chord_list = list(self.getChords()) # replace chords in note list
            self.expand() # expand tree with all combs
            self.fillPaths() # get all paths
            self.best() # compute best path
            self.writeFingers() # write new file
        if pr:
            print(self.prints)
     
    # ============================= ============================= #
    # fills the options dict with all possible fingering combinations
    # NEEDS: option dictionary from Options.py 
    def fillOptions(self):
        fingerings = OrderedDict()
        fingers = [0,1,2,3,4]
        for option in options:
            pos_comb = options[option]
            add_comb = ''
            # for every fret-string combination add a finger
            for comb in pos_comb.split(','):
                add_finger = ''
                for finger in fingers:
                    # CHECKING: open chord only when fret==0
                    if comb[0]!='0':
                        if finger!=0:
                            add_finger = '({0}):{1},{2}'.format(comb,finger,add_finger)
                    else:
                        if finger==0:
                            add_finger = '({0}):{1},{2}'.format(comb,finger,add_finger)
                add_finger = add_finger[:-1]
                add_comb = '{0},{1}'.format(add_finger,add_comb)
            add_comb = add_comb[:-1]
            fingerings[option]=add_comb
        return fingerings
        
    # ============================= ============================= #
    # fills note_list with the XML notes
    def fillNotes(self):
        # gather notes
        note_list = []
        dom = parse(self.file)
        notes = dom.getElementsByTagName("note")
        # rests don't have steps or alters, filter them out.
        notes = filter(lambda note: not self.is_rest(note), notes)
        #notes = filter(lambda note: not self.is_accidental(note), notes) # non accidentals 
        # fill list with all notes
        for note in notes:
            if note.parentNode.parentNode.getAttribute("id") != "P1":
                break
            # alter '1' means sharp
            # alter '-1' means flat
            alter = self.get_alter(note)
            pitch, octave, voice, duration, x, y, measure = self.get_step(note)
            if alter == '1':
                pitch = pitch + 's'
            if alter == '-1':
                pitch = pitch + 'f'
            new_note = Note(pitch, octave, voice, duration, x, y, measure)
            new_note.setFingers(self.fingering_options[new_note.me])
            note_list.append(new_note)
        print('Creating tree for {0} notes'.format(len(note_list)))
        self.prints = self.prints + 'Creating tree for {0} notes'.format(len(note_list))
        return note_list
    
    # replace chords in note list
    def getChords(self):
        note_list = list(self.allign())
        prevx = 0
        currx = 0
        chords = [] # the list of chords to return 
        #new_chord = Chord()
        chord_notes = [] # the notes
        for note in note_list:
            currx = note.x
            if currx != prevx:
                chords.append(Chord(chord_notes))
                chord_notes = [] # refresh chord
                chord_notes.append(note) # add new note
                prevx = note.x
            else:
                chord_notes.append(note)
        chords.append(Chord(chord_notes)) # last note
        return chords[1:]
    
    #fix voices note allignment
    def allign(self):
        return sorted(self.note_list)
    
    # ============================= ============================= #
    # creates all possible fingering combinations
    def expand(self):
        pos = 0
        # create root
        n = Note('ROOT','0')
        n.setFingers('(0/0):0')
        self.root = NoteNode(n, self.rules, False, n.fingers)
        #self.root.score = -1000
        # continue for the rest notes
        stack = []
        stack.append(self.root)
        stack_dummy = [] # dummy stack
        lim = self.limit # set limit
        print('Child number limit set to: {0}'.format(lim))
        c = 0 # for limit
        msg = 0 # for printing
        i = 0 # for status bar
        meas_change = False
        meas_curr = self.chord_list[0].notes[0].measure
        for chord in self.chord_list: # start for all notes
            if meas_curr != chord.notes[0].measure:
                meas_curr = chord.notes[0].measure
                meas_change = True
            while stack:
                #sc = [] # scores
                stack_dummy = []
                for child in stack: # start for all new added nodes
                    c = 0 
                    #ch_finger = int(str(child.finger).split(',')[-1])
                    if len(chord.notes) == 1: # chord containing one note
                        note = chord.notes[0]
                        if len(note.fingers.split(','))>1:
                            for finger in note.fingers.split(','):
                                node = NoteNode(note, self.rules, False, finger, child)
                                if (c<lim): # check the scores are -lim
                                    stack_dummy.append(node)
                                    stack_dummy = sorted(stack_dummy)
                                    #sc.append(node.score)
                                    c += 1
                                elif stack_dummy[-1].score > node.score:
                                    stack_dummy[-1].replaceNodeData(node)
                                    stack_dummy = sorted(stack_dummy)
                                #elif max(sc)>node.score: # replace max score
                                    #stack_dummy[sc.index(max(sc))].replaceNodeData(node)
                                    #sc[sc.index(max(sc))] = node.score
                                else: 
                                    node.delete()
                        else:
                            node = NoteNode(note, self.rules, False, note.fingers, child)
                            if (c<lim):
                                stack_dummy.append(node)
                                stack_dummy = sorted(stack_dummy)
                                #sc.append(node.score)
                                c += 1
                            elif stack_dummy[-1].score > node.score:
                                stack_dummy[-1].replaceNodeData(node)
                                stack_dummy = sorted(stack_dummy)
                            #elif max(sc)>node.score: # replace max score
                                #stack_dummy[sc.index(max(sc))].replaceNodeData(node)
                                #sc[sc.index(max(sc))] = node.score
                            else: 
                                node.delete()
                    else: # actual chord with multiple notes
                        node = NoteNode(chord.notes, self.rules, True, parent = child)
                        stack_dummy.append(node)
                        c += 1
                stack = list(stack_dummy)
                #gc.collect()
                break # out of while with stack containing the leafs to be expanded
            msg += 1
            i += 1 # printing bars and stuff
            sys.stdout.write('\r')
            sys.stdout.write("[{: <50}] {}%".format(u'\u2713'*int((50/len(self.chord_list))*i), int((100/len(self.chord_list))*i)))
            sys.stdout.flush()
            #sleep(0.25)
            if (msg%5==0) or (meas_change==True): # status updates and limiting
            #if meas_change==True:
                meas_change = False
                #print('Expanded {0} notes... paths: {1}'.format(msg, len(stack)))
                self.prints = self.prints + '\nExpanded {0} notes... paths: {1}'.format(msg, len(stack))
                best = self._limit(stack)
                stack = list(best)
                #print('Cutting down to... {0}'.format(len(stack)))
                self.prints = self.prints + '\nCutting down to... {0}'.format(len(stack))
        self.leafs = list(stack)
        print('\nCreated tree with {0} leafs'.format(len(self.leafs)))
        self.prints = self.prints + '\nCreated tree with {0} leafs'.format(len(self.leafs))
    
    # ============================= ============================= #
    # fill fingering_paths set with all possible fingering paths
    # start from leafs
    def fillPaths(self):
        print('Computing paths')
        for node in self.leafs:
            path = ''
            curr = node
            while curr != self.root:
                path = curr.me + path
                path = ' > ' + path
                curr = curr.parent
            self.fingering_paths.add(path[3:])
            
        #print('Computing paths')
        for node in self.leafs:
            path = list()
            curr = node
            while not curr.equals(self.root):
                path.insert(0, curr)
                curr = curr.parent
            self._fingering_paths.add(tuple(path)) 
        #print('There were {0} possible paths'.format(len(self._fingering_paths)))
        #self.prints = self.prints + '\nThere were {0} possible paths'.format(len(self._fingering_paths))
        
    # ============================= ============================= #
    def score(self):
        # creates the scores dictionary
        for node in self._fingering_paths:
            self.scores[node] = node[-1].score
    
    def _limit(self, stack):
        # downsize stack list of nodes
        minv = stack[0].score
        i = 0
        while i<len(stack):
            if len(stack)==1:
                break
            if stack[i].score>minv:
                stack.pop(i)
                i = 1
                continue
            else: # stack[i].score<minv:
                minv = stack[i].score
                #for j in range(i-1):
                #    stack.pop(j)
                stack.pop(i-1)
                i = 1
                continue
        return stack
        
    def best(self):
        self.score()
        best = min(self.scores, key=self.scores.get)
        s = ''
        for node in best:
            s += node.me
            s += ' '
            if len(str(node.finger).split(','))>1: # split chords fingers
                #print(node.finger)
                for finger, note in zip(str(node.finger).split(','),node.note):
                    self.best_path.append(int(finger))
                    self.best_dict[note.xy] = int(finger)
            else:
                self.best_dict[node.note.xy] = node.finger
                self.best_path.append(node.finger)
        #print('Chosen path: {0} with score: {1}'.format(s[:-1], self.scores[best]))
        self.combs = s[:-1]
        self.prints = self.prints + '\nChosen path: {0} \nwith score: {1}'.format(s[:-1], self.scores[best])
        
    # ============================= ============================= #
    # print calls
    # print all the nodes in the tree DEPTH first
    def printTree(self, start = None):
        start = start or self.root
        if (start.equals(self.root)):
            print('Starting from root: \n{0}'.format(start.getNodeInfo()))
        if start.children:
            for ch in start.children:
                print(ch.getNodeInfo())
                self.printTree(ch)
    
    # print all fingering paths
    def printPaths(self):
        for path in self.fingering_paths:
            yield(path + '\n')
    
    # print all scored paths
    def printScores(self):
        for path in self._fingering_paths:
            yield(self.scores[path])
    
    # print all notes in the tree
    def printNoteList(self):
        for note in self.note_list:
            print(note.getNoteInfo())
        
    # ============================= ============================= #
    def validate(self, file):
        if os.path.isfile(file):
            return True
        else:
            self.prints = 'File does not exist. Exited.'
    
    # ============================= ============================= #
    # write computed fingerings to new XML file
    def writeFingers(self):
        tree = ET.parse(self.file)
        root = tree.getroot()
        combs = self.combs.split()
        p = 0 # count 
        # part 1- guitar, not the tablature
        for part in root.iter('part'):
            for meas in part.iter('measure'):
                measure = meas.get('number')
                if part.get('id')=="P1":
                    for child in meas.iter('note'):
                        acc = ''
                        # if it is not a rest, write fingering
                        if child.find('rest')==None:
                            if child.find('pitch/alter')!=None:
                                if child.find('pitch/alter').text == '1':
                                    acc = 's'
                                elif child.find('pitch/alter').text == '-1':
                                    acc = 'f'
                            pitch = child.find('pitch/step').text
                            pitch = pitch+acc
                            octave = child.find('pitch/octave').text
                            voice = child.find('voice').text
                            x = child.get('default-x')
                            y = child.get('default-y')
                            if child.find('duration'):
                                n = Note(pitch, octave, voice, child.find('duration').text, x, y, measure) #(pitch,octave,voice,duration,x,y,measure)
                            else:
                                n = Note(pitch, octave, voice, '0', x, y, measure)
                            # new element nodes
                            notations = ET.Element('notations')
                            technical = ET.SubElement(notations, 'technical') 
                            fingering = ET.SubElement(technical, 'fingering')
                            #fingering.text = str(self.best_path[p]) # old pathing
                            fingering.text = str(self.best_dict[n.xy])
                            child.append(notations)
                # write tablature
                # not working - index out of range
                elif part.get('id')=="P2":
                    for child in meas.iter('note'):
                        if child.find('notations/technical/string')!=None:
                            string = combs[p].split(',')[0][1:] 
                            fret = combs[p].split(',')[1][:-1]
                            child.find('notations/technical/string').text = string
                            child.find('notations/technical/fret').text = fret
                            p += 1
        self.new_file = str('New_{0}'.format(self.file))
        tree.write(self.new_file)
        print('New file writing complete')
            
    # ============================= ============================= #
    # XML parsing helping functions            
    def get_step(self, note):
        stepNode = note.getElementsByTagName("step")[0]
        octave = note.getElementsByTagName("octave")[0]
        voice = note.getElementsByTagName("voice")[0]
        x = note.getAttribute("default-x")
        y = note.getAttribute("default-y")
        measure = note.parentNode.getAttribute("number")
        if note.getElementsByTagName("duration"):
            duration = note.getElementsByTagName("duration")[0]
            return str(stepNode.childNodes[0].nodeValue), str(octave.childNodes[0].nodeValue), str(voice.childNodes[0].nodeValue), str(duration.childNodes[0].nodeValue), x, y, measure
        else:
            return str(stepNode.childNodes[0].nodeValue), str(octave.childNodes[0].nodeValue), str(voice.childNodes[0].nodeValue), '0', x, y, measure

    def get_alter(self, note):
        alters = note.getElementsByTagName("alter")
        if len(alters) == 0:
            return None
        return alters[0].childNodes[0].nodeValue

    def is_rest(self, note):
        return len(note.getElementsByTagName("rest")) > 0

    def is_accidental(self, note):
        return self.get_alter(note) != None

# ============================= ============================= ============================= #
# ============================= ============================= ============================= #  

In [ ]:
from Rules import rules
tree = NoteTree('aaa.xml', rules, 5)#False)

Creating tree for 700 notes
Child number limit set to: 5
[✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓ ] 99%

In [82]:
from Rules import rules
tree = NoteTree('tt.xml', rules, 5)#False)

Creating tree for 410 notes
Child number limit set to: 5
[✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓ ] 99%
Created tree with 1 leafs
Computing paths
New file writing complete
Creating tree for 410 notes
Expanded 5 notes... paths: 3125
Cutting down to... 1
Expanded 6 notes... paths: 5
Cutting down to... 1
Expanded 10 notes... paths: 625
Cutting down to... 1
Expanded 11 notes... paths: 5
Cutting down to... 1
Expanded 15 notes... paths: 625
Cutting down to... 1
Expanded 20 notes... paths: 3125
Cutting down to... 1
Expanded 22 notes... paths: 25
Cutting down to... 1
Expanded 25 notes... paths: 125
Cutting down to... 1
Expanded 27 notes... paths: 25
Cutting down to... 1
Expanded 30 notes... paths: 125
Cutting down to... 1
Expanded 32 notes... paths: 25
Cutting down to... 1
Expanded 35 notes... paths: 125
Cutting down to... 1
Expanded 36 notes... paths: 5
Cutting down to... 1
Expanded 40 notes... paths: 625
Cutting down to... 1
Expanded 43 notes... paths: 125
Cutting down to... 1
Expa

In [2]:
#export tablature
tab_tree = ET.parse('tt.xml')
root = tab_tree.getroot()
combs = ''
for part in root.iter('part'):
    for meas in part.iter('measure'):
        measure = meas.get('number')
        if part.get('id')=="P2":
            for child in meas.iter('note'):
                if child.find('notations/technical/string')!=None:
                    string = child.find('notations/technical/string').text
                    fret = child.find('notations/technical/fret').text
                    comb = '({0},{1})'.format(fret,string)
                    combs = combs + ' ' + comb

In [11]:
combs = '(5,4) (5,3) (5,4) (5,3) (5,4) (5,3) (5,3) (4,3) (2,3) (0,3) (1,2) (2,4) (3,4) (4,4) (0,3) (1,3) (2,3) (1,4) (0,4) (3,5) (0,5) (5,3) (8,2) (5,3) (8,2) (5,3) (8,2) (8,2) (7,2) (5,2) (3,2) (3,1) (0,2) (1,2) (2,2) (3,2) (4,2) (0,1) (3,3) (2,3) (0,3) (2,4) (0,3) (2,2) (3,2) (3,2) (2,3) (0,3) (3,4) (0,4) (3,5) (3,4) (1,3) (2,3) (2,3) (1,4) (0,4) (3,5) (0,5) (2,4) (1,2) (5,4) (5,2) (2,4) (1,2) (5,4) (5,2) (2,4) (1,2) (3,4) (3,2) (4,4) (4,2) (5,4) (5,2) (1,2) (3,5) (3,3) (3,5) (2,3) (3,5) (1,3) (3,5) (1,2) (1,2) (3,2) (0,3) (4,4) (3,4) (0,3) (1,2) (1,2) (1,2) (2,4) (0,4) (1,2) (2,3) (4,3) (5,3) (3,4) (0,4) (3,5) (5,5) (0,5) (3,4) (3,4) (3,4) (3,4) (2,5) (0,5) (3,6) (5,5) (3,4) (0,4) (2,4) (0,3) (1,5) (3,6) (1,5) (0,5) (2,5) (3,5) (0,4) (2,5) (3,5) (0,5) (2,5) (3,5) (0,4) (2,5) (2,4) (3,5) (0,4) (2,5) (3,5) (0,4) (2,4) (3,5) (3,4) (0,4) (2,4) (3,5) (0,4) (2,4) (3,4) (0,4) (0,3) (2,4) (3,4) (0,4) (8,2) (8,1) (8,2) (8,1) (8,2) (8,1) (8,1) (7,1) (5,1) (8,2) (0,1) (1,2) (0,1) (3,2) (1,1) (4,2) (2,1) (5,2) (3,1) (4,1) (5,3) (5,1) (4,2) (4,4) (3,2) (1,2) (3,4) (2,3) (1,2) (4,2) (0,1) (0,1) (1,2) (0,2) (2,3) (4,4) (3,5) (2,3) (2,4) (2,3) (2,4) (2,3) (2,4) (2,3) (2,3) (0,3) (4,4) (2,4) (4,4) (4,3) (4,4) (4,3) (4,4) (4,3) (4,3) (2,3) (0,3) (4,4) (0,3) (2,3) (0,2) (1,2) (2,3) (0,2) (1,2) (3,2) (3,1) (3,2) (3,1) (3,2) (4,3) (3,2) (3,1) (4,3) (3,2) (3,1) (2,3) (1,2) (2,1) (0,3) (0,2) (0,1) (4,4) (2,3) (3,2) (0,3) (0,2) (0,1) (0,3) (0,2) (0,1) (0,3) (0,2) (0,1) (4,4) (2,3) (3,2) (2,4) (0,3) (1,2) (5,5) (4,4) (4,3) (2,4) (0,3) (1,2) (2,4) (0,3) (1,2) (2,4) (0,3) (1,2) (5,5) (4,4) (4,3) (3,5) (2,4) (2,3) (2,5) (0,4) (0,3) (0,3) (4,4) (2,4) (4,4) (0,3) (1,2) (2,3) (4,4) (0,3) (5,5) (0,4) (2,5) (0,3) (3,2) (0,4) (1,2) (0,4) (0,2) (0,4) (3,3) (0,4) (5,1) (7,4) (8,2) (7,4) (7,2) (7,4) (6,2) (7,4) (5,2) (5,1) (5,2) (5,1) (5,2) (5,1) (5,1) (3,1) (1,1) (0,1) (2,3) (1,1) (2,3) (1,1) (0,3) (0,1) (3,4) (3,2) (2,4) (1,2) (5,5) (3,4) (5,5) (3,4) (3,5) (2,4) (1,5) (0,4) (5,6) (3,5) (3,3) (3,3) (3,3) (3,3) (0,3) (2,3) (1,2) (1,4) (1,4) (0,4) (2,4) (3,4) (0,3) (2,4) (3,4) (0,4) (2,4) (3,4) (0,3) (0,4) (2,3) (3,4) (0,3) (2,4) (4,4) (2,3) (0,2) (1,2) (3,2) (0,2) (1,2) (2,3) (0,2) (1,2) (3,2) (0,1) (6,2) (7,3) (5,2) (6,2) (8,2) (8,1) (8,2) (8,1) (8,2) (8,1) (8,1) (7,1) (5,1) (3,1) (5,1) (5,1) (5,1) (3,1) (1,1) (0,2) (0,1) (1,2) (1,1) (1,2) (1,1) (1,2) (1,1) (0,2) (0,1) (2,3) (3,2) (0,3) (1,2) (1,2) (0,2) (2,3) (0,2) (1,2) (1,1) (3,2) (0,2) (1,2) (2,4) (1,2)'

In [12]:
tree_combs = '(0,3) (1,2) (0,3) (1,2) (0,3) (1,2) (1,2) (4,3) (2,3) (0,3) (1,2) (2,4) (3,4) (4,4) (0,3) (1,3) (2,3) (1,4) (0,4) (3,5) (0,5) (1,2) (3,1) (1,2) (3,1) (1,2) (3,1) (3,1) (2,1) (0,1) (3,2) (3,1) (4,3) (5,3) (2,2) (3,2) (4,2) (5,2) (3,3) (2,3) (5,4) (2,4) (0,3) (2,2) (3,2) (3,2) (2,3) (5,4) (3,4) (0,4) (3,5) (3,4) (1,3) (2,3) (2,3) (1,4) (0,4) (3,5) (0,5) (2,4) (1,2) (0,3) (0,1) (2,4) (1,2) (0,3) (0,1) (2,4) (1,2) (8,5) (7,3) (4,4) (4,2) (0,3) (0,1) (10,4) (3,5) (3,3) (3,5) (2,3) (3,5) (1,3) (3,5) (0,3) (5,3) (4,4) (3,4) (1,2) (3,2) (2,4) (0,3) (1,2) (0,4) (1,2) (1,2) (0,4) (1,2) (2,3) (0,2) (1,2) (3,4) (3,5) (0,4) (0,5) (3,4) (2,5) (3,4) (0,5) (3,6) (3,4) (3,4) (5,5) (3,6) (3,4) (5,5) (2,4) (0,3) (1,5) (1,5) (0,5) (2,5) (3,5) (0,4) (2,5) (3,5) (5,6) (2,5) (3,5) (0,4) (2,5) (2,4) (3,5) (0,4) (2,5) (3,5) (0,4) (2,4) (3,5) (3,4) (5,5) (2,4) (3,5) (0,4) (2,4) (3,4) (0,4) (0,3) (2,4) (3,4) (5,5) (3,1) (8,1) (3,1) (8,1) (3,1) (8,1) (8,1) (7,1) (5,1) (8,2) (5,2) (5,3) (5,2) (7,3) (6,2) (4,2) (2,1) (5,2) (3,1) (4,1) (5,3) (5,1) (4,2) (4,4) (3,2) (5,3) (3,4) (2,3) (5,3) (8,3) (0,1) (0,1) (1,2) (0,2) (2,3) (4,4) (3,5) (2,3) (2,4) (7,4) (2,4) (2,3) (2,4) (2,3) (2,3) (0,3) (4,4) (2,4) (4,4) (4,3) (4,4) (0,2) (4,4) (0,2) (0,2) (2,3) (5,4) (4,4) (0,3) (2,3) (0,2) (1,2) (2,3) (4,3) (1,2) (3,2) (8,2) (7,3) (8,2) (7,3) (4,3) (3,2) (3,1) (4,3) (3,2) (3,1) (2,3) (1,2) (2,1) (0,3) (0,2) (0,1) (4,4) (2,3) (3,2) (0,3) (0,2) (0,1) (0,3) (0,2) (0,1) (0,3) (0,2) (0,1) (4,4) (2,3) (3,2) (7,5) (5,4) (5,3) (5,5) (4,4) (4,3) (7,5) (5,4) (5,3) (7,5) (5,4) (5,3) (7,5) (5,4) (5,3) (5,5) (4,4) (4,3) (3,5) (2,4) (2,3) (2,5) (0,4) (0,3) (0,3) (4,4) (2,4) (4,4) (0,3) (1,2) (2,3) (4,4) (5,5) (0,3) (0,4) (7,6) (5,4) (12,4) (0,4) (1,2) (0,4) (0,2) (0,4) (3,3) (5,5) (5,1) (7,4) (3,1) (2,3) (2,1) (2,3) (1,1) (2,3) (0,1) (5,1) (0,1) (5,1) (5,2) (5,1) (5,1) (8,2) (6,2) (5,2) (7,4) (6,2) (7,4) (6,2) (0,3) (0,1) (3,4) (3,2) (7,5) (5,3) (5,5) (3,4) (5,5) (3,4) (3,5) (2,4) (1,5) (0,4) (5,6) (3,5) (3,3) (3,3) (3,3) (3,3) (5,4) (7,4) (5,3) (6,5) (6,5) (5,5) (7,5) (8,5) (5,4) (7,5) (8,5) (5,5) (7,5) (8,5) (0,3) (0,4) (2,3) (3,4) (0,3) (2,4) (4,4) (2,3) (4,3) (5,3) (3,2) (4,3) (5,3) (2,3) (0,2) (5,3) (7,3) (5,2) (6,2) (7,3) (5,2) (6,2) (8,2) (8,1) (3,1) (8,1) (3,1) (8,1) (8,1) (7,1) (5,1) (8,2) (5,1) (5,1) (5,1) (8,2) (6,2) (0,2) (0,1) (1,2) (1,1) (5,3) (6,2) (1,2) (1,1) (0,2) (0,1) (2,3) (3,2) (5,4) (5,3) (5,3) (4,3) (7,4) (4,3) (5,3) (6,2) (7,3) (4,3) (5,3) (7,5) (5,3)'

In [13]:
# find accuracy 
# (correctly predicted / total testing size) × 100%
corr = 0
for mine,tab in zip(tree_combs.split(),combs.split()):
    if mine == tab:
        corr += 1
acc = corr/410*100

In [14]:
print(acc)

61.21951219512195


## Stuff

In [78]:
for c in tree.chord_list:
    print(''.join('{} {} {}'.format(n.me,n.x,n.y) for n in c.notes))

E-2 75.18 -75.00
F-2 106.81 -70.00
G-2 138.44 -65.00
C-4 170.08 -15.00
B-3 201.71 -20.00
A-3 233.34 -25.00


In [79]:
# sanity checks
for p,c in zip(tree.fingering_paths, tree._fingering_paths):
    ss = ''
    for s in c:
        ss += s.me + ' > '
    print(ss[:-2])
    print(c[-1].score)

E-2: (0/6): 0 > F-2: (1/6): 1 > G-2: (3/6): 3 > C-4: (1/2): 1 > B-3: (0/2): 0 > A-3: (2/3): 1 
-225
E-2: (0/6): 0 > F-2: (1/6): 1 > G-2: (3/6): 3 > C-4: (1/2): 1 > B-3: (0/2): 0 > A-3: (7/4): 3 
-220
E-2: (0/6): 0 > F-2: (1/6): 1 > G-2: (3/6): 3 > C-4: (1/2): 1 > B-3: (0/2): 0 > A-3: (2/3): 3 
-215
E-2: (0/6): 0 > F-2: (1/6): 1 > G-2: (3/6): 3 > C-4: (1/2): 1 > B-3: (0/2): 0 > A-3: (7/4): 4 
-215
E-2: (0/6): 0 > F-2: (1/6): 1 > G-2: (3/6): 3 > C-4: (1/2): 1 > B-3: (0/2): 0 > A-3: (2/3): 2 
-226


In [86]:
a = tree.leafs
b = sorted(tree.leafs)

In [87]:
for aa,bb in zip(a,b):
    print(aa.score, bb.score)

-226 -226
-225 -225
-220 -220
-215 -215
-215 -215


## Testing

In [9]:
from Rules import rules
tree = NoteTree('test.xml', rules, 10, pr=False)

Creating tree for 38 notes
Child number limit set to: 10
[✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓] 100%
Created tree with 4 leafs
Computing paths
New file writing complete


In [10]:
from Rules import rules
tree = NoteTree('b1.xml', rules, pr=False)

Creating tree for 820 notes
Child number limit set to: 5
[✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓] 100%
Created tree with 25 leafs
Computing paths
New file writing complete


In [11]:
from Rules import rules
tree = NoteTree('b2.xml', rules, pr=False)

Creating tree for 576 notes
Child number limit set to: 5
[✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓] 100%
Created tree with 5 leafs
Computing paths
New file writing complete


In [2]:
from Rules import rules
tree = NoteTree('Dust4.xml', rules, pr=False)

Creating tree for 1125 notes
Child number limit set to: 5
[✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓] 100%
Created tree with 1 leafs
Computing paths
New file writing complete
